In [1]:
!pip install google-cloud-aiplatform protobuf==3.20.3 --upgrade --user

In [2]:
!pip install -U google-cloud-aiplatform "shapely<2"

In [3]:
from vertexai.language_models import TextGenerationModel
from vertexai.language_models import ChatModel
import os
from six.moves import urllib
import tempfile
import numpy as np
import pandas as pd
import tensorflow as tf

2024-05-22 13:58:39.894476: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 13:58:40.690982: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 13:58:40.691112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 13:58:40.840290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 13:58:41.182073: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 13:58:41.186719: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [38]:
# Pull in/import the clinical coding sample dataframe

# Storage directory
DATA_DIR = os.path.join(tempfile.gettempdir(), 'census_data')
#print(DATA_DIR)
tf.io.gfile.makedirs(DATA_DIR)

DATA_URL = (
    'https://raw.githubusercontent.com/garthajon/DataScienceColabRepo/main')
TRAINING_FILE = 'NCCS_CLINICAL_CODING_EXAMPLES_NOHEAD.csv'
TRAINING_URL = '%s/%s' % (DATA_URL, TRAINING_FILE)
#print(TRAINING_URL)

# These are the features in the dataset.

_CSV_COLUMNS = [
    'IDFIELD','NCCS_CLINICAL_TEXT','NCCS_PRIM_DIAG',
    'NCCS_PRIM_DIAG_DESC','NCCS_SEC_DIAG','NCCS_SEC_DIAG_DESC','NCCS_DOC_VERSION','PAGE,NOTES'   
]

training_file_path = os.path.join(DATA_DIR, TRAINING_FILE)

#print(training_file_path)
temp_file, _ = urllib.request.urlretrieve(TRAINING_URL)
with tf.io.gfile.GFile(temp_file, 'r') as temp_file_object:
    with tf.io.gfile.GFile(training_file_path, 'w') as file_object:
        for line in temp_file_object:
            line = line.strip()
            line = line.replace(', ', ',')
            if not line or ',' not in line:
                continue
            if line[-1] == '.':
                line = line[:-1]
            line += '\n'
            file_object.write(line)
tf.io.gfile.remove(temp_file)

df = pd.read_csv(training_file_path, names=_CSV_COLUMNS,
                           na_values='?')

df.head()


,IDFIELD,NCCS_CLINICAL_TEXT,NCCS_PRIM_DIAG,NCCS_PRIM_DIAG_DESC,NCCS_SEC_DIAG,NCCS_SEC_DIAG_DESC,NCCS_DOC_VERSION,"PAGE,NOTES"
1,Patient is admitted with cellulitis of the leg...,L97.X,"Ulcer of lower limb,not elsewhere classified",L03.1,Cellulitis of other parts of limb,ICD10 5TH EDITION MORBIDITY 2024 V10.1,35,NaN
2,"Patient admitted with slurred speech,right sid...",I60.1,Subarachnoid haemorrhage from middle cerebral ...,NaN,NaN,ICD10 5TH EDITION MORBIDITY 2024 V10.2,39,NaN
3,Patient admitted for treatment of a diabetic c...,H28.0,Diabetic cataract (E10E14 with common fourth c...,E10.3,"Type 1 diabetes mellitus,with ophthalmic compl...",ICD10 5TH EDITION MORBIDITY 2024 V10.3,41,DAGGER ASTERISK EXAMPLE
4,Retrobulbar neuritis in multiple sclerosis,G35.X,Multiple sclerosis,H48.1,Retrobulbar neuritis in diseases classified el...,ICD10 5TH EDITION MORBIDITY 2024 V10.4,42,DAGGER ASTERISK EXAMPLE
5,Urinary sepsis and biliary sepsis (responsible...,N39.0,"Urinary tract infection,site not specified",K83.0,Cholangitis,ICD10 5TH EDITION MORBIDITY 2024 V10.5,53,NaN


In [39]:
# add additional columns into the table to contain AI prompt-derived responses
df['Diag1_Derived'] = ''
df['Diag2_Derived'] = ''
df.head()

,IDFIELD,NCCS_CLINICAL_TEXT,NCCS_PRIM_DIAG,NCCS_PRIM_DIAG_DESC,NCCS_SEC_DIAG,NCCS_SEC_DIAG_DESC,NCCS_DOC_VERSION,"PAGE,NOTES",Diag1_Derived,Diag2_Derived
1,Patient is admitted with cellulitis of the leg...,L97.X,"Ulcer of lower limb,not elsewhere classified",L03.1,Cellulitis of other parts of limb,ICD10 5TH EDITION MORBIDITY 2024 V10.1,35,NaN,,
2,"Patient admitted with slurred speech,right sid...",I60.1,Subarachnoid haemorrhage from middle cerebral ...,NaN,NaN,ICD10 5TH EDITION MORBIDITY 2024 V10.2,39,NaN,,
3,Patient admitted for treatment of a diabetic c...,H28.0,Diabetic cataract (E10E14 with common fourth c...,E10.3,"Type 1 diabetes mellitus,with ophthalmic compl...",ICD10 5TH EDITION MORBIDITY 2024 V10.3,41,DAGGER ASTERISK EXAMPLE,,
4,Retrobulbar neuritis in multiple sclerosis,G35.X,Multiple sclerosis,H48.1,Retrobulbar neuritis in diseases classified el...,ICD10 5TH EDITION MORBIDITY 2024 V10.4,42,DAGGER ASTERISK EXAMPLE,,
5,Urinary sepsis and biliary sepsis (responsible...,N39.0,"Urinary tract infection,site not specified",K83.0,Cholangitis,ICD10 5TH EDITION MORBIDITY 2024 V10.5,53,NaN,,


In [5]:
# create the vertex (prompt AI model)
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [17]:
#value1 = df.at[1, 'IDFIELD']
#print(value1)

Patient is admitted with cellulitis of the leg and a severe leg ulcer.


In [43]:
# prompt engineering (using the one shot/one example approach)

# ONE SHOT PROMPT - SETTING OUT THE EXAMPLE FOR THE VERTEX AI TEXT MODEL

for idx in df.index:
    value1 = df.at[idx, 'IDFIELD']
    #print(value1)
    prompt1 = f"""Output a notation's primary and secondary ICD10 code.

    Notation: Patient is admitted with cellulitis of the leg and a severe leg ulcer
    ICD10 codes: L97.X, L03.1 

    Notation: {value1}
    ICD10 codes:
    """

    #print(prompt1)
    var_output = generation_model.predict(prompt=prompt1, max_output_tokens=256).text
    #var_output.replace(" ", "")
    print(var_output)
    prim_diag = var_output[0:5]
    sec_diag = var_output[7:]
    df.at[idx, 'Diag1_Derived'] = prim_diag
    df.at[idx, 'Diag2_Derived'] = sec_diag


#prompt = """Output a notation's primary and secondary ICD10 code.
#Notation: {value1}
#ICD10 codes: L97.X,L03.1
#Notation: Urinary sepsis and biliary sepsis (responsible consultant confirms UTI and cholangitis without
#sepsis)/n"+ 
#ICD10 codes:
#"""
#var_output = generation_model.predict(prompt=prompt, max_output_tokens=256).text

#print(var_output)

#prim_diag = var_output[0:5]

#print(prim_diag)

#sec_diag = var_output[7:]

df.head()



L97.X, L03.1
I60.0, I60.9
H25.9, E11.9
G35.0, H47.0
N13.9, K83.0


,IDFIELD,NCCS_CLINICAL_TEXT,NCCS_PRIM_DIAG,NCCS_PRIM_DIAG_DESC,NCCS_SEC_DIAG,NCCS_SEC_DIAG_DESC,NCCS_DOC_VERSION,"PAGE,NOTES",Diag1_Derived,Diag2_Derived
1,Patient is admitted with cellulitis of the leg...,L97.X,"Ulcer of lower limb,not elsewhere classified",L03.1,Cellulitis of other parts of limb,ICD10 5TH EDITION MORBIDITY 2024 V10.1,35,NaN,L97.X,L03.1
2,"Patient admitted with slurred speech,right sid...",I60.1,Subarachnoid haemorrhage from middle cerebral ...,NaN,NaN,ICD10 5TH EDITION MORBIDITY 2024 V10.2,39,NaN,I60.0,I60.9
3,Patient admitted for treatment of a diabetic c...,H28.0,Diabetic cataract (E10E14 with common fourth c...,E10.3,"Type 1 diabetes mellitus,with ophthalmic compl...",ICD10 5TH EDITION MORBIDITY 2024 V10.3,41,DAGGER ASTERISK EXAMPLE,H25.9,E11.9
4,Retrobulbar neuritis in multiple sclerosis,G35.X,Multiple sclerosis,H48.1,Retrobulbar neuritis in diseases classified el...,ICD10 5TH EDITION MORBIDITY 2024 V10.4,42,DAGGER ASTERISK EXAMPLE,G35.0,H47.0
5,Urinary sepsis and biliary sepsis (responsible...,N39.0,"Urinary tract infection,site not specified",K83.0,Cholangitis,ICD10 5TH EDITION MORBIDITY 2024 V10.5,53,NaN,N13.9,K83.0


In [45]:
# summarise findings visually
# Select columns 'a' and 'b' and rename them to 'col1' and 'col2'
new_df = df[['NCCS_CLINICAL_TEXT', 'NCCS_PRIM_DIAG_DESC','Diag1_Derived','Diag2_Derived']].rename(columns={'NCCS_CLINICAL_TEXT': 'NCCS_PRIM_DIAG_DESC', 'NCCS_PRIM_DIAG': 'NCCS_SEC_DIAG'})

# Print the new DataFrame
print(new_df)


  NCCS_PRIM_DIAG_DESC NCCS_PRIM_DIAG_DESC Diag1_Derived Diag2_Derived
1              L97.X               L03.1          L97.X         L03.1
2               I60.1                 NaN         I60.0         I60.9
3               H28.0               E10.3         H25.9         E11.9
4               G35.X               H48.1         G35.0         H47.0
5              N39.0               K83.0          N13.9         K83.0
